In [1]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from img2vec import rgb2emb
import math
import os
import pandas as pd
import joblib

In [2]:
# Define the batch size
batch_size = 64

### Read the data

In [3]:
train_data = pd.read_csv(os.path.join('..', '..', 'data', 'train.csv'))
val_data = pd.read_csv(os.path.join('..', '..', 'data', 'val.csv'))
test_data = pd.read_csv(os.path.join('..', '..', 'data', 'test.csv'))

### Add image paths

In [4]:
def construct_img_path(row):
    return os.path.join("..", "..", "data", "faces", row['user_id'],
                        "coarse_tilt_aligned_face." + str(row['face_id']) + "." + row['original_image'])

train_data['img_path'] = train_data.apply(construct_img_path, axis=1)
val_data['img_path'] = val_data.apply(construct_img_path, axis=1)
test_data['img_path'] = test_data.apply(construct_img_path, axis=1)


### Check if images exist

In [5]:
train_data['img_exists'] = train_data['img_path'].apply(os.path.exists)
val_data['img_exists'] = val_data['img_path'].apply(os.path.exists)
test_data['img_exists'] = test_data['img_path'].apply(os.path.exists)

### Filter out any rows where the image doesn't exist

In [6]:
train_data_filtered = train_data[train_data['img_exists'] == True]
val_data_filtered = val_data[val_data['img_exists'] == True]
test_data_filtered = test_data[test_data['img_exists'] == True]

### Encode age labels

In [7]:
age_encoder = LabelEncoder()
train_data['age_label'] = age_encoder.fit_transform(train_data['age'])
val_data['age_label'] = age_encoder.transform(val_data['age'])
test_data['age_label'] = age_encoder.transform(test_data['age'])
num_classes = len(age_encoder.classes_)
print("Age classes:", age_encoder.classes_)

Age classes: ['(0, 2)' '(15, 20)' '(25, 32)' '(38, 43)' '(4, 6)' '(48, 53)' '(60, 100)'
 '(8, 23)']


In [8]:
joblib.dump(age_encoder, 'age_encoder.pkl')
print("Age encoder saved successfully.")

Age encoder saved successfully.


### Save the encoder

### Extract image paths and labels

In [9]:
train_image_paths = train_data_filtered['img_path'].tolist()
train_labels = train_data_filtered['age_label'].values

val_image_paths = val_data_filtered['img_path'].tolist()
val_labels = val_data_filtered['age_label'].values

test_image_paths = test_data_filtered['img_path'].tolist()
test_labels = test_data_filtered['age_label'].values

KeyError: 'age_label'

### Define function that process features in batches and store them to avoid recomputation

In [ ]:
def preprocess_and_save_features(image_paths, output_file, batch_size=64):
    if os.path.exists(output_file):
        print(f"Loading pre-processed features from {output_file}")
        return np.load(output_file)
    
    print(f"Processing {len(image_paths)} images and saving to {output_file}")
    all_features = []
    
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}/{math.ceil(len(image_paths)/batch_size)}")
        batch_features = rgb2emb(batch_paths)
        all_features.append(batch_features)
    
    all_features = np.vstack(all_features)
    np.save(output_file, all_features)
    return all_features

### Process and save features

In [ ]:
train_features = preprocess_and_save_features(train_image_paths, 'train_features.npy')
val_features = preprocess_and_save_features(val_image_paths, 'val_features.npy')
test_features = preprocess_and_save_features(test_image_paths, 'test_features.npy')


### Standardize features

In [ ]:
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

### Save the scaler

In [ ]:
joblib.dump(scaler, 'feature_scaler.pkl')

### Define a simple softmax model 

In [ ]:
model = Sequential([
    Dense(num_classes, activation='softmax', input_shape=(train_features.shape[1],))
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

### Train the model

In [ ]:
print("Training the model...")
history = model.fit(
    train_features_scaled, train_labels,
    validation_data=(val_features_scaled, val_labels),
    epochs=30,
    batch_size=batch_size,
)

### Save the model

In [ ]:
model.save('softmax_age_classifier.h5')
print("Model saved successfully.")

### Evaluate the model

In [ ]:
print("Evaluating the model on test data...")
test_loss, test_acc = model.evaluate(test_features_scaled, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

### Get predictions

In [ ]:
test_predictions = model.predict(test_features_scaled)
test_pred_classes = np.argmax(test_predictions, axis=1)

### Visualize results

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

# Confusion matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(test_labels, test_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=age_encoder.classes_,
            yticklabels=age_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()

### Classification report

In [ ]:
print("Classification Report:")
print(classification_report(test_labels, test_pred_classes, target_names=age_encoder.classes_))


### Example of using the model

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('softmax_age_classifier.h5')
print("Model loaded successfully.")

# Load the LabelEncoder
age_encoder = joblib.load('age_encoder.pkl')
print("Age encoder loaded successfully.")

# Load the scaler
scaler = joblib.load('feature_scaler.pkl')
print("Feature scaler loaded successfully.")

def predict_age(image_path, model, age_encoder, scaler):
    """
    Predict the age range for a given face image.
    
    Parameters:
    image_path (str): Path to the image file
    model: Trained Keras model
    age_encoder: Trained LabelEncoder for age classes
    scaler: Trained StandardScaler for feature normalization
    
    Returns:
    tuple: (predicted_age_range, confidence)
    """
    # Check if file exists
    if not os.path.exists(image_path):
        print(f"Error: File {image_path} not found")
        return None, 0
        
    # Extract features
    print(f"Extracting features from {image_path}...")
    features = rgb2emb([image_path])
    
    # Check if feature extraction was successful
    if features.size == 0:
        print("Error: Feature extraction failed")
        return None, 0
        
    # Standardize features
    print("Standardizing features...")
    features_scaled = scaler.transform(features)
    
    # Make prediction
    print("Making prediction...")
    pred_probs = model.predict(features_scaled, verbose=0)[0]
    
    # Get predicted class
    pred_class = np.argmax(pred_probs)
    
    # Convert to age range
    pred_age_range = age_encoder.classes_[pred_class]
    confidence = pred_probs[pred_class]

    return pred_age_range, confidence

# Example usage with a sample image
# Replace with an actual path to test
sample_image_path = test_image_paths[0]  # Using the first test image as an example
print(f"Using sample image: {sample_image_path}")

# Make prediction
pred_age, confidence = predict_age(sample_image_path, model, age_encoder, scaler)

if pred_age is not None:
    print(f"Predicted age range: {pred_age} with confidence {confidence:.2f}")
    
    # Display the image if possible
    try:
        from PIL import Image
        img = Image.open(sample_image_path)
        plt.figure(figsize=(4, 4))
        plt.imshow(img)
        plt.title(f"Predicted: {pred_age} (Confidence: {confidence:.2f})")
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Could not display image: {e}")

print("\nCode for making predictions on new images:")